<a href="https://colab.research.google.com/github/SrikarR63/Phishing_Website_Detection_Using_ML_and_NLP_Models_TARP_Project_/blob/main/GPT2_Phishing_Website_Detection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install -q git+https://github.com/huggingface/transformers.git
# Install helper functions.
!pip install -q git+https://github.com/gmihaila/ml_things.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
#phish_data = pd.read_csv('/content/phishing_site_urls.csv') #reading the csv file
url="https://drive.google.com/file/d/1WI3hdAFaVi3UcjMPyWPlI1ttkBDNTXKv/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
phish_data = pd.read_csv(path)

In [4]:
phish_good = []
phish_bad = []
for i in range(0,len(phish_data['Label'])):
  if phish_data['Label'].iloc[i] == 'good':
    phish_good.append(phish_data['URL'].iloc[i])
  else:
    phish_bad.append(phish_data['URL'].iloc[i])

len(phish_good)

392924

In [5]:
len(phish_bad)

156422

In [6]:
import io
import os
import torch
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from ml_things import plot_dict, plot_confusion_matrix, fix_text
from sklearn.metrics import classification_report, accuracy_score
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW,
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification)

# Set seed for reproducibility.
set_seed(123)

# Number of training epochs (authors on fine-tuning Bert recommend between 2 and 4).
epochs = 4

# Number of batches - depending on the max sequence length and GPU memory.
# For 512 sequence length batch of 10 works without cuda memory issues.
# For small sequence length can try batch of 32 or higher.
batch_size = 32

# Pad or truncate text sequences to a specific length
# if `None` it will use maximum sequence of word piece tokens allowed by model.
max_length = 60

# Look for gpu to use. Will use `cpu` by default if no gpu found.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Name of transformers model - will use already pretrained model.
# Path of transformer model - will load your own model from local disk.
model_name_or_path = 'gpt2'

# Dictionary of labels and their id - this will be used to convert.
# String labels to number ids.
labels_ids = {'good': 0, 'bad': 1}

# How many labels are we using in training.
# This is used to decide size of classification head.
n_labels = len(labels_ids)

/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/dtypes.py:35: DeprecationWarning: ml_dtypes.float8_e4m3b11 is deprecated. Use ml_dtypes.float8_e4m3b11fnuz
  from tensorflow.tsl.python.lib.core import pywrap_ml_dtypes


In [7]:
n_labels

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2

Load Model and Tokenizer

Loading the three essential parts of the pretrained GPT2 transformer: configuration, tokenizer and model.

For this example I will use gpt2 from HuggingFace pretrained transformers. In creating the model_config I will mention the number of labels I need for my classification task. Since I only predict two sentiments: positive and negative I will only need two labels for num_labels.

Creating the tokenizer is pretty standard when using the Transformers library. After creating the tokenizer it is critical for this tutorial to set padding to the left tokenizer.padding_side = "left" and initialize the padding token to tokenizer.eos_token which is the GPT2's original end of sequence token. This is the most essential part of this tutorial since GPT2 uses the last token for prediction so we need to pad to the left.

HuggingFace already did most of the work for us and added a classification layer to the GPT2 model. In creating the model I used GPT2ForSequenceClassification. Since we have a custom padding token we need to initialize it for the model using model.config.pad_token_id. Finally we will need to move the model to the device we defined earlier.

In [8]:
# Get model configuration.
print('Loading configuraiton...')
model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path=model_name_or_path, num_labels=n_labels)

# Get model's tokenizer.
print('Loading tokenizer...')
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path=model_name_or_path)
# default to left padding
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token = 50256
tokenizer.pad_token = tokenizer.eos_token


# Get the actual model.
print('Loading model...')
model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name_or_path, config=model_config)

# resize model embedding to match new tokenizer
model.resize_token_embeddings(len(tokenizer))

# fix model padding token id
model.config.pad_token_id = model.config.eos_token_id

# Load model to defined device.
model.to(device)
print('Model loaded to `%s`'%device)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Loading configuraiton...
Loading tokenizer...
Loading model...


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded to `cpu`


Dataset and Collator This is where I create the PyTorch Dataset and Data Loader with Data Collator objects that will be used to feed data into our model. Since we need to input numbers to our model we need to convert the texts and labels to numbers. This is the purpose of a collator.

In [1]:
!pip install fast_ml --quiet

In [9]:
from fast_ml.model_development import train_valid_test_split

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
phish_good

["esxcc.com/js/index.htm?us.battle.net/noghn/en/?ref5M+9d'bbws&b`|tNEfnud&d\xad\rÅj\x8fYÖJ\x15p\x94\x92lR+CmP \x0c!\x10¡6\x96Ô~PÖtÓ\x9e¶P8~·Äëi\x0f\x12q°ôß|±½$~.ë½\x1fü·ÝjQ³dì÷VTø\x04\x06srDiG[\x80v\x15' éò\x12å¯ê\x02YÃ\x8eò\x84\x7f«Ù¾FkþàâFV#àÐØ\x94}Ú\x0b\x83¨\x08%¡\x9a\x90x(My¼RçÁs\t²¹ù#Õ\x15¦kI\x11çzb\x7fTUùØ9\x88ÂEm© ÁqÚ'lMJ\x92!c<E\x8a§æèÉ§¹\x82ü\x85el#ñ\xad\x9e·Låß¯¥=\x9b\x12\x97ºxÌ\x03üHÖÞþ¼¸^'](6¼\x1a!¦\x117jôu\xa0´GÔiP°Û¶\x9cÆ±È\x01>9o¼°iìTvµD\x83Ã%&£.\x90k\x10&Ï\x9dXO\x1d\x81O¾O©ªØ¤ÉÂ0¾cÄ²¿\rHi±æáÐ\\\x1a/'\x9b¡!¿F#\t?ì.:\x15â±¾«\x10æD\x85ô\x841]Íµ¬6\xa0\x8bãê¤G;\x81Ê\x16ö2°ï¾øPü·\x8b}9÷ð\x99IP=\x86â\n\x8eVz3¹ó¬éíA\x08\x8c vwK\x8c}·Áªí_S¢D\x07\x837NØØòa0úqÞ\x96ñ\x0c\x98f½tjÓµ»w2xhçt\x9f\\I^u\x96\x80Æ\x8a\x9c!¯ì½ú\xa0°S\x01CYïõJï£l\x1ad\x19:6Õ±Ç4\x04\x08»ê\x1bRùú\x11#Òuü@^6\x96amp;offerId=newmail-en-us-v2&amp;siteState=ver:4",
 "www\x0eeira¯&nvinip¿ncH¯wVö%ÆåyDaHðû/ÏyEùu\x03Ë\nÓ\x176(rTÃ\x06u\x0f\x8f\x7fæ\x82\x0c\x99=g\x810¾\x96÷mÖi\x12Ó-;\x9bXZ\\%êýü\x05Éfn&\x87\\°%7õÙ:¹u\x96\

In [11]:
phish_bad

['nobell.it/70ffb52d079109dca5664cce6f317373782/login.SkyPe.com/en/cgi-bin/verification/login/70ffb52d079109dca5664cce6f317373/index.php?cmd=_profile-ach&outdated_page_tmpl=p/gen/failed-to-load&nav=0.5.1&login_access=1322408526',
 'www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrcmd=_home-customer&nav=1/loading.php',
 'serviciosbys.com/paypal.cgi.bin.get-into.herf.secure.dispatch35463256rzr321654641dsf654321874/href/href/href/secure/center/update/limit/seccure/4d7a1ff5c55825a2e632a679c2fd5353/',
 'mail.printakid.com/www.online.americanexpress.com/index.html',
 'thewhiskeydregs.com/wp-content/themes/widescreen/includes/temp/promocoessmiles/?84784787824HDJNDJDSJSHD//2724782784/',
 'smilesvoegol.servebbs.org/voegol.php',
 'premierpaymentprocessing.com/includes/boleto-2via-07-2012.php',
 'myxxxcollection.com/v1/js/jih321/bpd.com.do/do/l.popular.php',
 'super1000.info/docs',
 'horizonsgallery.com/js/bin/ssl1/_id/www.paypal.com/fr/cgi-bin/webscr/cmd=_registration-run/login.php?cmd=_login-run&am

In [12]:
gooddf=pd.read_csv("/content/gdrive/MyDrive/good.csv")
baddf=pd.read_csv("/content/gdrive/MyDrive/bad.csv")

In [13]:
gooddf.tail(15)

,URL,Label
392815,wfhm.com/loans/danny-fitzpatrick/about.page,good
392816,wfhm.com/loans/heather-phipps,good
392817,wfhm.com/wfhm/danny-fitzpatrick/,good
392818,whitworth.edu/Athletics/Teams/BasketballMen/Co...,good
392819,wirelessrefill.com/,good
392820,worldswithoutend.com/books_Locus-F_index.asp,good
392821,worldswithoutend.com/books_Locus-SF_index.asp,good
392822,wreathsacrossamerica.org/locations/,good
392823,xing.com/profile/Nils_Hagemann,good
392824,youthleaguesusa.com/potomacsoccer/11-12/Tryout...,good


In [14]:
baddf.head(3)

,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad


In [52]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 2.0 MB/s eta 0:00:00


In [53]:
!pip install transformers[torch]

In [54]:
!pip install transformers
!pip install accelerate


In [55]:
pip install --upgrade transformers

In [15]:
# Add a new pad token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Set the pad_token to the newly added token
tokenizer.pad_token = '[PAD]'


**TRAINING THE MODEL**

In [16]:
import pandas as pd
import torch

# Loading our dataset (assuming 'good.csv' and 'bad.csv' containg data)


# Combine the two datasets into one
combined_data = pd.concat([gooddf, baddf], ignore_index=True)

# Tokenize the URL data
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
# Add a new pad token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Set the pad_token to the newly added token
tokenizer.pad_token = '[PAD]'

# Encode the URLs and convert labels to numerical IDs
input_ids = []
attention_masks = []
labels = []

for index, row in combined_data.iterrows():
    url = row['URL']
    label = row['Label']

    # Encode the URL text
    inputs = tokenizer(url, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')
    input_ids.append(inputs['input_ids'])
    attention_masks.append(inputs['attention_mask'])

    # Convert labels to numerical IDs
    label_id = labels_ids[label]
    labels.append(label_id)

# Convert lists to PyTorch tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels, dtype=torch.long)

# Define your dataset class
from torch.utils.data import TensorDataset
dataset = TensorDataset(input_ids, attention_masks, labels)

# Define your data loader
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
#validdf=pd.read_csv("/content/drive/MyDrive/validation_phishing_dataset.csv")

In [ ]:
#validdf.head(5)

**VALIDATION DATASET**

In [17]:
goodvdf=pd.read_csv("/content/gdrive/MyDrive/latestvalidation_good.csv")
badvdf=pd.read_csv("/content/gdrive/MyDrive/latestvalidation_bad.csv")

In [ ]:
goodvdf=pd.read_csv("/content/gdrive/MyDrive/gd_urlv.csv")
badvdf=pd.read_csv("/content/gdrive/MyDrive/bd_urlv.csv")

In [18]:
import pandas as pd
import torch

# Load your dataset (assuming 'good.csv' and 'bad.csv' contain your data)


# Combine the two datasets into one
combined_data = pd.concat([goodvdf, badvdf], ignore_index=True)

# Tokenize the URL data
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
# Add a new pad token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Set the pad_token to the newly added token
tokenizer.pad_token = '[PAD]'

# Encode the URLs and convert labels to numerical IDs
input_ids = []
attention_masks = []
labels = []

for index, row in combined_data.iterrows():
    url = row['URL']
    label = row['Label']

    # Encode the URL text
    inputs = tokenizer(url, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')
    input_ids.append(inputs['input_ids'])
    attention_masks.append(inputs['attention_mask'])

    # Convert labels to numerical IDs
    label_id = labels_ids[label]
    labels.append(label_id)

# Convert lists to PyTorch tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels, dtype=torch.long)

# Define your dataset class
from torch.utils.data import TensorDataset
dataset = TensorDataset(input_ids, attention_masks, labels)

# Define your data loader
from torch.utils.data import DataLoader
val_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [19]:
goodvdf.head(5)

,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,good
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,good
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,good
3,mail.printakid.com/www.online.americanexpress....,good
4,thewhiskeydregs.com/wp-content/themes/widescre...,good


In [20]:
badvdf.head(5)

,URL,Label
0,users.chariot.net.au/~geastwood/,bad
1,www.nucleus.com/~eureka/,bad
2,www.aero.com/bonaire/,bad
3,bellsouthpwp.net/a/v/avifliflight/,bad
4,www.northlandcollege.edu/programs/aviation_mai...,bad


In [21]:
# Add a new pad token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Set the pad_token to the newly added token
tokenizer.pad_token = '[PAD]'

**Predict URL Category Function**

In [63]:
#val_df=pd.read_csv("/content/drive/MyDrive/mixedvalid.csv")

In [22]:
tokenizer.pad_token = '[PAD]'

In [65]:
!pip install transformers[torch]
!pip install accelerate

In [69]:
pip install accelerate -U

In [23]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
#from datasets import load_dataset

# Load your labeled dataset
import pandas as pd

# Replace 'your_dataset.csv' with the actual file path or URL of your dataset
dataset_path = "/content/gdrive/MyDrive/validation_phishing_dataset.csv"

# Load the dataset into a Pandas DataFrame
dataset = pd.read_csv(dataset_path)

# Now, you can work with the 'df' DataFrame to access and manipulate your data.
# For example, you can access specific columns, perform data analysis, and more.

# Example: Accessing the first few rows of the DataFrame
print(dataset.head())

# Example: Filter URLs with 'good' label
good_urls = dataset[dataset['Label'] == 'good']

# Example: Filter URLs with 'bad' label
bad_urls = dataset[dataset['Label'] == 'bad']

# Load a pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # You can choose a different variant if needed
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2ForSequenceClassification.from_pretrained(model_name)

# Define a training and evaluation data collate function
def collate_fn(examples):
    inputs = tokenizer(examples['URL'], padding='max_length', truncation=True, max_length=128)
    labels = examples['Label']
    return {
        'input_ids': inputs.input_ids,
        'attention_mask': inputs.attention_mask,
        'labels': labels
    }

# Prepare data for training and evaluation
train_dataset = dataset[dataset['Label'] == 'good']
eval_dataset = dataset[dataset['Label'] == 'bad']

# Define training arguments
training_args = TrainingArguments(
    output_dir='./url_classification',
    evaluation_strategy='steps',
    eval_steps=1000,
    save_steps=1000,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    save_total_limit=2,
    learning_rate=2e-5,
)

# Create a Trainer instance and fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

#trainer.train()

# Perform inference
#url = "https://example.com"
url = "http://shadetreetechnology.com/V4/validation/a"
url=input("Enter the url to be checked: ")
input_ids = tokenizer(url, return_tensors='pt').input_ids
output = model(input_ids)
predicted_label = torch.argmax(output.logits, dim=1).item()
res=""
if predicted_label==0:
  res="good"
else:
  res="bad"

#print(f"Predicted label for {url}: {predicted_label}")
print(f"Predicted label for {url}: {res}")

                                                 URL Label
0              http://www.crestonwood.com/router.php  good
1  http://shadetreetechnology.com/V4/validation/a...   bad
2  https://support-appleld.com.secureupdate.duila...   bad
3                                 http://rgipt.ac.in  good
4  http://www.iracing.com/tracks/gateway-motorspo...  good


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Enter the url to be checked: docs.google.com/spreadsheet/viewform?formkey=dE5rVEdSV2pBdkpSRy11V3o2eDdwbnc6MQ
Predicted label for docs.google.com/spreadsheet/viewform?formkey=dE5rVEdSV2pBdkpSRy11V3o2eDdwbnc6MQ: good


**URL CLASSIFICATION**

In [25]:
!pip install transformers

In [34]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

In [24]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2ForSequenceClassification.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

urls=[]
ch='y'
# Example URLs
"""while ch=='y':
  u=input()
  urls.append(u)
  ch=input('Do you want to continue?')
"""
urls = [
    "https://www.example.com",
    "https://www.notlegit.com",
    "vtop.vit.ac.in",
    "http://phc.prontonetworks.com/cgi-bin/authlogin?URI=http://www.msftconnecttest.com/redirect"
]

# Tokenize and classify URLs
for url in urls:
    inputs = tokenizer(url, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    prediction = torch.argmax(probabilities, dim=-1).item()

    label = "Phishing" if prediction == 1 else "Legitimate"
    print(f"URL: {url} - Classification: {label}")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


URL: https://www.example.com - Classification: Legitimate
URL: https://www.notlegit.com - Classification: Phishing
URL: vtop.vit.ac.in - Classification: Legitimate
URL: http://phc.prontonetworks.com/cgi-bin/authlogin?URI=http://www.msftconnecttest.com/redirect - Classification: Legitimate
